In [ ]:
print 'hello world!'
import sys
sys.version

hello world!


'2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 17:05:42) \n[GCC 7.2.0]'

In [5]:
# Imports the Google Cloud client library
#from google.cloud import storage
from pyspark.sql import SparkSession
from itertools import combinations
from collections import Counter
from scipy.sparse import coo_matrix, save_npz
import numpy as np
import datetime
import logging

logging.basicConfig(filename='spark-assignment.log',level=logging.DEBUG)
logging.warning('this is the beginning of the log file')

spark = SparkSession \
    .builder \
    .appName("PySpark Intro") \
    .getOrCreate()
    
bucket_dir = 'gs://bgse-datawarehousing-random-tweets'
file1_name = 'gs://bgse-datawarehousing-random-tweets/2019-02-26T00:00:30.657Z'

In [3]:
##Functions
def rdd_transforms(files):

    tweets = files.rdd.map(lambda x: x.entities).filter(lambda x: x is not None) \
        .map(lambda x: x.hashtags).filter(lambda x: x is not None) \
        .filter(lambda x: len(x) > 1).map(lambda x: [i[1] for i in x]) \
        .map(lambda x: [i.lower() for i in x]).cache()

    pairs =  tweets.flatMap(lambda x: sorted(list(combinations(x,2)))).cache()
    tags = tweets.flatMap(lambda x: [i for i in x]).distinct().cache()

    counts = pairs.map(lambda x: [x,1]).reduceByKey(lambda a,b: a + b).cache()

    dict_fromTag = tags.zipWithIndex().collectAsMap()
    
    return counts,dict_fromTag

def output_sparse_vectors(counts,dict_fromTag):
    myrows = []
    mycols = []
    mydata = []

    myrows = counts.map(lambda x: dict_fromTag[x[0][0]]).collect() + \
        counts.map(lambda x: dict_fromTag[x[0][1]]).collect()
    mycols = counts.map(lambda x: dict_fromTag[x[0][1]]).collect() + \
        counts.map(lambda x: dict_fromTag[x[0][0]]).collect()

    mydata = counts.map(lambda x: x[1]).collect()
    mydata = mydata + mydata
    
    return myrows, mycols, mydata

def print_runningtime(start_time):
    print('%s minutes elapsed.' % str((datetime.datetime.now()-start_time).total_seconds()/60))

In [ ]:
#Prod

start_time = datetime.datetime.now()
print(start_time)
logging.warning('Beginning run at %s' % (datetime.datetime.now()))

files = spark.read.options(samplingRatio=0.01).json(bucket_dir)
print_runningtime(start_time)
logging.warning('Finished loading files at %s' % datetime.datetime.now())

counts, dict_fromTag = rdd_transforms(files)
print_runningtime(start_time)
logging.warning('Finished rdd_transforms at %s' % datetime.datetime.now())

myrows, mycols, mydata = output_sparse_vectors(counts, dict_fromTag)
print_runningtime(start_time)
logging.warning('Finished outputting sparse vectors at %s' % datetime.datetime.now())

mat = coo_matrix((mydata,(myrows,mycols)),shape=(len(dict_fromTag),len(dict_fromTag)))
print_runningtime(start_time)
print(datetime.datetime.now())
logging.warning('Finished creating matrix at %s' % datetime.datetime.now())

save_npz("output_mat_final.npz", mat, compressed=True)

print_runningtime(start_time)
print(datetime.datetime.now())
logging.warning('Finished saving sparse matrix at %s' % datetime.datetime.now())

2019-10-30 20:18:20.308611
2.4507813 minutes elapsed.
40.5972096 minutes elapsed.
43.9376648833 minutes elapsed.
43.9473593833 minutes elapsed.
2019-10-30 21:02:17.150356
44.0061819333 minutes elapsed.
2019-10-30 21:02:20.679769
